In [2]:
import getpass
import os
from langchain_cohere import ChatCohere
os.environ["COHERE_API_KEY"] = getpass.getpass()
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import FewShotChatMessagePromptTemplate

 ········


In [10]:
from langchain_chroma import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_core.example_selectors import LengthBasedExampleSelector
from langchain_cohere import CohereEmbeddings

In [4]:
chat_model = ChatCohere()

In [14]:
examples = [
    {"question": "من عاشق این فیلمم!", 
     "answer":  "مثبت"},
    {"question": "از این صندلی خوشم نمی‌اد",
     "answer":  "منفی"}
    ]

In [15]:
example_prompt = ChatPromptTemplate.from_messages(
    [("human", "{question}"),
     ("ai", "{answer}")
    ]
)

In [17]:
without_few_shot = ChatPromptTemplate.from_messages(
    [(  "system", "احساس متن زیر رو دسته بندی کن"),
        ("human", "{question}")
    ]
)

chain_without_few_shot = without_few_shot | chat_model

chat_ai = chain_without_few_shot.invoke({"question": "آخه کی می‌ره این فیلمو ببینه؟"})
print(chat_ai.content)

متن ارائه‌شده بیانگر احساسات منفی و انتقاد نسبت به یک فیلم است. فردی که این متن را نوشته، احتمالاً احساس ناامیدی یا نارضایتی از آن فیلم دارد و تمایلی به تماشای آن نشان نمی‌دهد. این جمله می‌تواند بیانگر طنز یا تمسخر نیز باشد، اما در کل، احساس غالب در این متن، عدم علاقه و انتقاد است.


In [18]:
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt_with_few_shot = ChatPromptTemplate.from_messages(
    [(  "system", "احساس متن زیر رو دسته بندی کن"),
     few_shot_prompt,
     ("human", "{question}")
    ]
)

In [19]:
chain_with_few_shot = final_prompt_with_few_shot | chat_model

chat_ai = chain_with_few_shot.invoke({"question": "آخه کی می‌ره این فیلمو ببینه؟"})

print(chat_ai.content)

منفی


## Lecture 2

In [20]:
examples = [
    {
        "question": "خوشحال",
        "answer": "ناراحت"
    },
    {
        "question": "مادر",
        "answer": "مهربان"
    },
    {
        "question": "روز",
        "answer": "خورشید"
    },
]

embeddings = CohereEmbeddings(model='embed-multilingual-v3.0')
to_vectorize = [" ".join(example.values()) for example in examples]
vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=examples)

In [21]:
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2,
)

In [22]:
example_selector.select_examples({"question": "مضطرب"})

[{'answer': 'ناراحت', 'question': 'خوشحال'},
 {'answer': 'ناراحت', 'question': 'خوشحال'}]

In [23]:
few_shot_prompt = FewShotChatMessagePromptTemplate(
    input_variables=["question"],
    example_selector=example_selector,
    example_prompt=ChatPromptTemplate.from_messages(
        [("human", "{question}"), ("ai", "{answer}")]
    ),
)

final_prompt = ChatPromptTemplate.from_messages(
    [
     few_shot_prompt,
     ("human", "{question}")
    ]
)

chain = final_prompt | chat_model

chat_ai = chain.invoke({"question": "مضطرب"})
print(chat_ai.content)

chat_ai = chain.invoke({"question": "پدر"})
print(chat_ai.content)

همراه
قوی


## Lecture 3

In [24]:
example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length= 25)

dynamic_prompt = FewShotChatMessagePromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    input_variables=["question"]
)

KeyError: 'example_prompt'